# batch running alos2App.py

Firstly, let's specify the folders

In [ ]:
import os
from pathlib import Path

# home folder for batch
home_dir = Path('/data/alos2/isce_process')
# folder containing alos2 data
alos_data_dir = home_dir / 'alos_data'
# folder to save interferograms
ifgs_dir = home_dir / 'ifgs'

# folder contains aux data
aux_dir = home_dir / 'aux_data'

dem1_dir = aux_dir / 'dem_1_arcsec'
dem3_dir = aux_dir / 'dem_3_arcsec'
wb_dir = aux_dir / 'wbd_1_arcsec'

Specify the ifg pairs

In [ ]:
pars = [
    ['210919', '220114'],
    ['210919', '220123'],
    ['210919', '211212'],
    ['210919', '211031'],
]


Specify the configs of 'alos2App.xml' file.

In [ ]:
str = '''<?xml version="1.0" encoding="UTF-8"?>
<alos2App>
  <component name="alos2insar">

    <property name="reference directory">{}</property>
    <property name="secondary directory">{}</property>

    <property name="dem for coregistration">{}</property>
    <property name="dem for geocoding">{}</property>
    <property name="water body">{}</property>

    <property name="geocode bounding box">[38.4573,39.0911,98.6675,99.3582]</property>

    <property name="interferogram filter strength">{}</property> 
    <property name="interferogram filter window size">{}</property>

    <property name="number of range looks ion">40</property>
    <property name="number of azimuth looks ion">16</property>
  </component>
</alos2App>
'''

Now, let's batch running alos2App.py

In [ ]:
fs, ws = 0.1, 3
for p in pars:
    ref, sec = p
    folder_ref = alos_data_dir / ref
    folder_sec = alos_data_dir / sec

    name = f'{ref}-{sec}'
    work_dir = ifgs_dir / name

    work_dir.mkdir(exist_ok=True, parents=True)

    os.chdir(work_dir)

    # link aux
    os.system(f'ln -s {aux_dir}/* ./')

    dem1 = list(aux_dir.joinpath('dem_1_arcsec').glob('*.wgs84'))[0]
    dem3 = list(aux_dir.joinpath('dem_3_arcsec').glob('*.wgs84'))[0]
    wbd1 = list(aux_dir.joinpath('wbd_1_arcsec').glob('*.wbd'))[0]

    xml = str.format(folder_ref, folder_sec, dem1,
                     dem3, wbd1,
                     fs, ws)
    xml_file = work_dir / 'alos2App.xml'
    with open(xml_file, 'w') as f:
        f.write(xml)

    print(f'\n==============  {ref}-{sec} ==============')

    os.system('alos2App.py --start=startup')
